In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [63]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
import os

from sklearn.preprocessing import MinMaxScaler
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, r2_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

In [4]:
df = pd.read_csv('cleaned_data.csv')
pd.set_option('display.max_columns', None)
df

,Unnamed: 0,index,SiteInfor,Country,Latitude,Longitude,Elevation,Duration,SamplingDepth,SamplingThickness,SoilBD,SandPerc,SiltPerc,Texture,SoilpH,BackgroundSOC,SoilFamily,CoverCrop,CoverCropGroup,GrainCrop,GrainCropGroup,Conservation_Type,ControlDescription,No_C,No_T
0,527,527,Centrralia in Missouri,USA,39.216667,-92.116667,262.2,15.0,0-to-10,10,1.28,5.255556,56.966667,Silt,5.344444,0.811111,Mesic vertic albaqualfs,Red_colver,Legume,Corn/soybean/wheat,CSW,CC-NT,No cover crop,3.0,3.0
1,528,528,Centrralia in Missouri,USA,39.216667,-92.116667,262.2,15.0,10-to-20,10,1.28,5.255556,56.966667,Silt,5.344444,0.811111,Mesic vertic albaqualfs,Red_colver,Legume,Corn/soybean/wheat,CSW,CC-NT,No cover crop,3.0,3.0
2,529,529,Centrralia in Missouri,USA,39.216667,-92.116667,262.2,15.0,20-to-30,10,1.28,5.255556,56.966667,Silt,5.344444,0.811111,Mesic vertic albaqualfs,Red_colver,Legume,Corn/soybean/wheat,CSW,CC-NT,No cover crop,3.0,3.0
3,809,809,Kentucky Agricultural Experiment Station Far...,USA,38.030000,-84.510000,299.6,5.0,0-to-20,20,1.05,7.300000,70.200000,Silt loam,5.480000,1.160000,Mesic Typic Paleudalf,Rye,Rye,Corn,Corn,CC,Fallow,2.0,2.0
4,810,810,Kentucky Agricultural Experiment Station Far...,USA,38.030000,-84.510000,299.6,5.0,0-to-20,20,1.05,7.300000,70.200000,Silt loam,5.480000,1.160000,Mesic Typic Paleudalf,Rye,Rye,Corn,Corn,CC,Fallow,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
455,4959,4959,"Las Torres-Tomejil Farm, Seville, Spain",Spain,38.141667,-5.266667,601.0,11.0,0-to-15,15,1.34,28.500000,45.800000,Loam,8.200000,0.759800,Xeroﬂuvent,Vegetal compost,NaN,Bean-beet-carrot-chard-cucumber-marrow,AVG,OF,No description,12.0,12.0
456,5143,5143,"Sjaelland, Denmark",Denmark,55.460000,11.720000,46.0,46.0,6-to-13,7,1.54,71.000000,14.000000,Loamy sand,6.700000,1.624000,No soil family,Grass-clover,AVG,Average of oat Barley,AVG,OF-CC,No description,3.0,3.0
457,5144,5144,"Sjaelland, Denmark",Denmark,55.460000,11.720000,46.0,46.0,6-to-13,7,1.54,65.000000,21.000000,Sandy loam,6.400000,1.450000,No soil family,Grass-clover,AVG,Average of oat Barley,AVG,OF-CC,No description,3.0,3.0
458,5145,5145,"Sjaelland, Denmark",Denmark,55.460000,11.720000,46.0,47.0,6-to-13,7,1.35,59.000000,20.000000,Sandy loam,7.100000,2.030000,No soil family,Grass-clover,AVG,Average of oat Barley,AVG,OF-CC,No description,9.0,9.0


# Transform categorical data to numerical data

In [8]:
# check the textual content of column
df['SamplingDepth'].value_counts()

# map to numerical data

df['SamplingDepth']=df['SamplingDepth'].map({'0-to-20':10,'0-to-15':7.5,'0-to-40':20,'0-to-25':12.5,'0-to-5':2.5,'0-to-10':5,
                                             '0-to-45':22.5,'0-to-30':15,'161-to-300':230.5,'61-to160':110.5,'10-to-20':15,
                                             '20-to-30':22.5,'10-to-30':20,'15-to-30':22.5,
                                            '5-to-15':10,'20-to-40':30,'6-to-13':9.5,'15-to-20':17.5,
                                            '5-to--20':22.5,'10-to-15':12.5,'5-to-10':7.5})

0-to-20       192
0-to-15        51
0-to-40        25
0-to-25        24
0-to-5         23
0-to-10        18
0-to-45        17
0-to-30        17
161-to-300     12
0-to-60        12
61-to160       12
10-to-20        9
20-to-30        9
10-to-30        9
15-to-30        6
5-to-15         5
20-to-40        5
6-to-13         4
15-to-20        4
5-to-20         2
10-to-15        2
5-to-10         2
Name: SamplingDepth, dtype: int64

In [10]:
df['SamplingDepth'].value_counts()

10.0     197
7.5       53
20.0      34
22.5      32
15.0      26
12.5      26
2.5       23
5.0       18
110.5     12
230.5     12
30.0       5
17.5       4
9.5        4
Name: SamplingDepth, dtype: int64

In [11]:
# check column Texture, and decide to use one-hot encoding (dummy)
df['Texture'].value_counts()

Silt loam          209
Sandy loam          97
Loam                73
Sandy clay          43
Sandy clay loam     16
Clay                 9
Silty clay loam      6
Silt                 3
Silty clay           3
Loamy sand           1
Name: Texture, dtype: int64

In [12]:
# use ordinal encoding

df['SoilFamily'].value_counts()

# transform to ordinal data
encoder = OrdinalEncoder()
df['SoilFamily'] = encoder.fit_transform(df[['SoilFamily']])

No soil family                                                                                                 123
Mesic Typic Paleudalf                                                                                           74
Norfolk sandy loam (fine-loamy, siliceous, thermic, Typic Kandiudults)(FAO classification: Orthic Luvisols)     54
Kandiustalf                                                                                                     43
Haplustalfs                                                                                                     30
Typic Hapludult                                                                                                 24
Typic Xerofluvent loam soil                                                                                     24
Aridic Argiborolls                                                                                              12
Dothan sandy loam (fine-loamy, siliceous, thermic, Plinthic, Kandiudults)       

In [13]:
# use one-hot encoding
df['Conservation_Type'].value_counts()

CC       287
AF       104
CC-NT     37
OF-CC     22
OF        10
Name: Conservation_Type, dtype: int64

In [14]:
# use one-hot encoding
df['ControlDescription'].value_counts()

No cover crop                          289
Fallow                                 115
Continuous maize without fertilizer     43
No description                          13
Name: ControlDescription, dtype: int64

In [6]:
# one-hot encoding for non-ordinal columns
def onehot_encode(data, columns):
    for column in columns:
        dummies = pd.get_dummies(data[column])
        data = pd.concat([data,dummies], axis=1)
        data.drop(column,axis=1,inplace=True)
    return data

In [16]:
onehot_columns = (['Texture', 'Conservation_Type', 'ControlDescription'])
df = onehot_encode(df, onehot_columns)

KeyError: 'Texture'

In [28]:
df

,Unnamed: 0,index,SiteInfor,Country,Latitude,Longitude,Elevation,Duration,SamplingDepth,SamplingThickness,SoilBD,SandPerc,SiltPerc,SoilpH,BackgroundSOC,SoilFamily,CoverCrop,CoverCropGroup,GrainCrop,GrainCropGroup,No_C,No_T,Clay,Loam,Loamy sand,Sandy clay,Sandy clay loam,Sandy loam,Silt,Silt loam,Silty clay,Silty clay loam,AF,CC,CC-NT,OF,OF-CC,Continuous maize without fertilizer,Fallow,No cover crop,No description
0,527,527,Centrralia in Missouri,USA,39.216667,-92.116667,262.2,15.0,5.0,10,1.28,5.255556,56.966667,5.344444,0.811111,9.0,Red_colver,Legume,Corn/soybean/wheat,CSW,3.0,3.0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0
1,528,528,Centrralia in Missouri,USA,39.216667,-92.116667,262.2,15.0,15.0,10,1.28,5.255556,56.966667,5.344444,0.811111,9.0,Red_colver,Legume,Corn/soybean/wheat,CSW,3.0,3.0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0
2,529,529,Centrralia in Missouri,USA,39.216667,-92.116667,262.2,15.0,22.5,10,1.28,5.255556,56.966667,5.344444,0.811111,9.0,Red_colver,Legume,Corn/soybean/wheat,CSW,3.0,3.0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0
3,809,809,Kentucky Agricultural Experiment Station Far...,USA,38.030000,-84.510000,299.6,5.0,10.0,20,1.05,7.300000,70.200000,5.480000,1.160000,8.0,Rye,Rye,Corn,Corn,2.0,2.0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0
4,810,810,Kentucky Agricultural Experiment Station Far...,USA,38.030000,-84.510000,299.6,5.0,10.0,20,1.05,7.300000,70.200000,5.480000,1.160000,8.0,Rye,Rye,Corn,Corn,2.0,2.0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
455,4959,4959,"Las Torres-Tomejil Farm, Seville, Spain",Spain,38.141667,-5.266667,601.0,11.0,7.5,15,1.34,28.500000,45.800000,8.200000,0.759800,18.0,Vegetal compost,NaN,Bean-beet-carrot-chard-cucumber-marrow,AVG,12.0,12.0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1
456,5143,5143,"Sjaelland, Denmark",Denmark,55.460000,11.720000,46.0,46.0,9.5,7,1.54,71.000000,14.000000,6.700000,1.624000,10.0,Grass-clover,AVG,Average of oat Barley,AVG,3.0,3.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1
457,5144,5144,"Sjaelland, Denmark",Denmark,55.460000,11.720000,46.0,46.0,9.5,7,1.54,65.000000,21.000000,6.400000,1.450000,10.0,Grass-clover,AVG,Average of oat Barley,AVG,3.0,3.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1
458,5145,5145,"Sjaelland, Denmark",Denmark,55.460000,11.720000,46.0,47.0,9.5,7,1.35,59.000000,20.000000,7.100000,2.030000,10.0,Grass-clover,AVG,Average of oat Barley,AVG,9.0,9.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1


In [ ]:
df.isna().sum()

In [ ]:
df['SamplingDepth'].fillna(df['SamplingDepth'].mean(), inplace=True)

# scale all the data into -1 to 1

In [ ]:
from sklearn.preprocessing import StandardScaler

In [68]:
sc = StandardScaler()
features = sc.fit_transform(df_drop)
features_scaler = pd.DataFrame(features, columns=df_drop.columns, index=df_drop.index)
features_scaler.head()

,Latitude,Longitude,Elevation,Duration,SamplingDepth,SamplingThickness,SoilBD,SandPerc,SiltPerc,SoilpH,BackgroundSOC,SoilFamily,No_C,No_T,Clay,Loam,Loamy sand,Sandy clay,Sandy clay loam,Sandy loam,Silt,Silt loam,Silty clay,Silty clay loam,AF,CC,CC-NT,OF,OF-CC,Continuous maize without fertilizer,Fallow,No cover crop,No description
0,0.521853,-1.055728,-0.329629,1.650577,-0.395867,-0.650384,-0.024397,-1.366965,0.549653,-1.013000,-0.564860,-0.034793,-0.139276,-0.139276,-0.141264,-0.434316,-0.046676,-0.321119,-0.189832,-0.516931,12.342339,-0.912507,-0.081022,-0.11496,-0.540495,-1.288006,3.381188,-0.149071,-0.224117,-0.321119,-0.577350,0.769217,-0.170537
1,0.521853,-1.055728,-0.329629,1.650577,-0.134302,-0.650384,-0.024397,-1.366965,0.549653,-1.013000,-0.564860,-0.034793,-0.139276,-0.139276,-0.141264,-0.434316,-0.046676,-0.321119,-0.189832,-0.516931,12.342339,-0.912507,-0.081022,-0.11496,-0.540495,-1.288006,3.381188,-0.149071,-0.224117,-0.321119,-0.577350,0.769217,-0.170537
2,0.521853,-1.055728,-0.329629,1.650577,0.061873,-0.650384,-0.024397,-1.366965,0.549653,-1.013000,-0.564860,-0.034793,-0.139276,-0.139276,-0.141264,-0.434316,-0.046676,-0.321119,-0.189832,-0.516931,12.342339,-0.912507,-0.081022,-0.11496,-0.540495,-1.288006,3.381188,-0.149071,-0.224117,-0.321119,-0.577350,0.769217,-0.170537
3,0.468225,-0.964939,-0.244155,-0.291278,-0.265084,-0.250040,-1.371175,-1.277739,1.116368,-0.890634,-0.317108,-0.277289,-0.950248,-0.950248,-0.141264,-0.434316,-0.046676,-0.321119,-0.189832,-0.516931,-0.081022,1.095882,-0.081022,-0.11496,-0.540495,0.776394,-0.295754,-0.149071,-0.224117,-0.321119,1.732051,-1.300022,-0.170537
4,0.468225,-0.964939,-0.244155,-0.291278,-0.265084,-0.250040,-1.371175,-1.277739,1.116368,-0.890634,-0.317108,-0.277289,-0.950248,-0.950248,-0.141264,-0.434316,-0.046676,-0.321119,-0.189832,-0.516931,-0.081022,1.095882,-0.081022,-0.11496,-0.540495,0.776394,-0.295754,-0.149071,-0.224117,-0.321119,1.732051,-1.300022,-0.170537


# slice & split dataset into training and test datasets

In [ ]:
cover_crop = df['CoverCrop']
cover_cropgroup = df['CoverCropGroup']
grain_crop = df['GrainCrop']
grain_cropgroup = df['GrainCropGroup']

In [56]:
# slice the df, ignore the columns 'index' & 'country'

df_1 = df.iloc[:, 4:]

drop_list = ['CoverCrop','CoverCropGroup','GrainCrop','GrainCropGroup']
df_drop = df_1.drop(drop_list, axis=1)

df_drop

,Latitude,Longitude,Elevation,Duration,SamplingDepth,SamplingThickness,SoilBD,SandPerc,SiltPerc,SoilpH,BackgroundSOC,SoilFamily,No_C,No_T,Clay,Loam,Loamy sand,Sandy clay,Sandy clay loam,Sandy loam,Silt,Silt loam,Silty clay,Silty clay loam,AF,CC,CC-NT,OF,OF-CC,Continuous maize without fertilizer,Fallow,No cover crop,No description
0,39.216667,-92.116667,262.2,15.0,5.0,10,1.28,5.255556,56.966667,5.344444,0.811111,9.0,3.0,3.0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0
1,39.216667,-92.116667,262.2,15.0,15.0,10,1.28,5.255556,56.966667,5.344444,0.811111,9.0,3.0,3.0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0
2,39.216667,-92.116667,262.2,15.0,22.5,10,1.28,5.255556,56.966667,5.344444,0.811111,9.0,3.0,3.0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0
3,38.030000,-84.510000,299.6,5.0,10.0,20,1.05,7.300000,70.200000,5.480000,1.160000,8.0,2.0,2.0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0
4,38.030000,-84.510000,299.6,5.0,10.0,20,1.05,7.300000,70.200000,5.480000,1.160000,8.0,2.0,2.0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
455,38.141667,-5.266667,601.0,11.0,7.5,15,1.34,28.500000,45.800000,8.200000,0.759800,18.0,12.0,12.0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1
456,55.460000,11.720000,46.0,46.0,9.5,7,1.54,71.000000,14.000000,6.700000,1.624000,10.0,3.0,3.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1
457,55.460000,11.720000,46.0,46.0,9.5,7,1.54,65.000000,21.000000,6.400000,1.450000,10.0,3.0,3.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1
458,55.460000,11.720000,46.0,47.0,9.5,7,1.35,59.000000,20.000000,7.100000,2.030000,10.0,9.0,9.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1


In [57]:
# split into training and test

x_train, x_test, y_train, y_test = train_test_split(df_drop, cover_crop, test_size = 0.2, random_state = 11)

In [69]:
# split with scaled data
x_train_sc, x_test_sc, y_train_sc, y_test_sc = train_test_split(features_scaler, cover_crop, test_size = 0.2, random_state = 11)

# build models --- Random Forest Tree

In [72]:
rf = RandomForestClassifier(random_state=0)
rf.fit(x_train,y_train)

y_preds_rf=rf.predict(x_test)
print("Training Accuracy is:", rf.score(x_train,y_train))
print("Test Accuracy is:", rf.score(x_test, y_test))

Training Accuracy is: 0.8315217391304348
Test Accuracy is: 0.6956521739130435


In [65]:
print('Accuracy Score : ' + str(accuracy_score(y_test,y_preds_rf)))
print('Precision Score : ' + str(precision_score(y_test,y_preds_rf,average='macro')))
print('Recall Score : ' + str(recall_score(y_test,y_preds_rf,average='macro')))
print('F1 Score : ' + str(f1_score(y_test,y_preds_rf,average='macro')))
# print('Roc area is : ' + str(roc_auc_score(y_test, y_predict_proba_rfc, multi_class='ovr')))

Accuracy Score : 0.6956521739130435
Precision Score : 0.41975308641975306
Recall Score : 0.42407407407407405
F1 Score : 0.41419753086419747


In [73]:
# model with scaled data


rf_scale = RandomForestClassifier(random_state=0)
rf_scale.fit(x_train_sc,y_train_sc)

y_preds_sc=rf_scale.predict(x_test_sc)
print("Training Accuracy is:", rf.score(x_train_sc,y_train_sc))
print("Test Accuracy is:", rf.score(x_test_sc, y_test_sc))

Training Accuracy is: 0.3233695652173913
Test Accuracy is: 0.3804347826086957
